# Applied AI Midterm — SRGAN + Classifiers (PyTorch)
Complete runnable notebook template for the midterm. Edit dataset paths and hyperparameters where marked.

## 0. Setup
Edit DATA_ROOT and run cells. Install packages if needed: `!pip install timm torchmetrics` in Colab.

In [2]:
# Cell: Imports and Globals
# TODO: set DATA_ROOT to your dataset
DATA_ROOT = 'C:/Users/lolze/Documents/Github/Midterm_AppliedAI/data/raw'  # <-- change me
PROCESSED_128 = 'C:/Users/lolze/Documents/Github/Midterm_AppliedAI/data/processed_128'
LOWRES_32 = 'C:/Users/lolze/Documents/Github/Midterm_AppliedAI/data/lowres_32'
SEED = 42
IMG_CHANNELS = 3

# TODO: run rest of the notebook cells to execute the pipeline

## 1. Prepare datasets (resize & save)
This cell will resize images to 128x128 and 32x32 and save to folders.

In [3]:
# Cell: prepare_datasets (run once)
from PIL import Image
import os, shutil
def prepare_datasets(root, out_hr, out_lr, size_hr=128, size_lr=32, force=False):
    if force:
        shutil.rmtree(out_hr, ignore_errors=True)
        shutil.rmtree(out_lr, ignore_errors=True)
    os.makedirs(out_hr, exist_ok=True)
    os.makedirs(out_lr, exist_ok=True)
    classes = [d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]
    classes.sort()
    for cls in classes:
        in_dir = os.path.join(root, cls)
        out_hr_cls = os.path.join(out_hr, cls); os.makedirs(out_hr_cls, exist_ok=True)
        out_lr_cls = os.path.join(out_lr, cls); os.makedirs(out_lr_cls, exist_ok=True)
        for fname in os.listdir(in_dir):
            if not fname.lower().endswith(('.png','.jpg','.jpeg')): continue
            src = os.path.join(in_dir, fname)
            img = Image.open(src).convert('RGB')
            img.resize((size_hr,size_hr), Image.BICUBIC).save(os.path.join(out_hr_cls, fname))
            img.resize((size_lr,size_lr), Image.BICUBIC).save(os.path.join(out_lr_cls, fname))
prepare_datasets(DATA_ROOT, PROCESSED_128, LOWRES_32, force=False)

## 2. Dataloaders
Create ImageFolder datasets and DataLoaders for HR and LR images.

In [3]:
# Cell: dataloaders
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
mean = [0.485,0.456,0.406]; std=[0.229,0.224,0.225]
transform_hr = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
transform_lr = transforms.Compose([transforms.ToTensor()])
hr_dataset = datasets.ImageFolder(PROCESSED_128, transform=transform_hr)
lr_dataset = datasets.ImageFolder(LOWRES_32, transform=transform_lr)
print('HR samples', len(hr_dataset), 'LR samples', len(lr_dataset))

HR samples 25000 LR samples 25000


## 3. Classifier A — Transfer learning
This section contains a full training loop using timm EfficientNet-B0.

In [4]:
# Cell: classifier A - Transfer learning (Complete Implementation)
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import os
from datetime import datetime
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create model
modelA = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2)
modelA = modelA.to(device)

# Create data loaders
dataset_size = len(hr_dataset)
train_size = int(0.7 * dataset_size)
val_size = dataset_size - train_size

train_dataset, val_dataset = random_split(hr_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelA.parameters(), lr=0.0001)

# Setup directories
os.makedirs('models', exist_ok=True)
init_time = datetime.now()
current_time = init_time.strftime('%Y%m%d_%H%M%S')
name_dir = f'models/classifier_A_{current_time}'
os.makedirs(name_dir, exist_ok=True)

writer = SummaryWriter(log_dir=os.path.join(name_dir, 'logs'))

# Training loop
best_val_accuracy = 0.0
patience = 10
patience_counter = 0
epochs = 50

for epoch in range(epochs):
    # Training
    modelA.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = modelA(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    
    train_accuracy = 100 * train_correct / train_total
    train_loss /= len(train_loader)
    
    # Validation
    modelA.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_labels = []
    all_predictions = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = modelA(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            
            # Store for metrics
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            probs = torch.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy()[:, 1])  # Probability for class 1
    
    val_accuracy = 100 * val_correct / val_total
    val_loss /= len(val_loader)
    
    # Calculate additional metrics
    val_f1 = f1_score(all_labels, all_predictions, average='binary')
    val_auc = roc_auc_score(all_labels, all_probs)
    
    print(f'Epoch {epoch+1}/{epochs}:')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
    print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')
    print(f'  Val F1: {val_f1:.4f}, Val AUC: {val_auc:.4f}')
    
    # Tensorboard logging
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('Accuracy/train', train_accuracy, epoch)
    writer.add_scalar('Accuracy/val', val_accuracy, epoch)
    writer.add_scalar('F1/val', val_f1, epoch)
    writer.add_scalar('AUC/val', val_auc, epoch)
    
    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        patience_counter = 0
        torch.save(modelA.state_dict(), os.path.join(name_dir, 'classifier_A_best.pth'))
        print(f'  -> New best model saved! Accuracy: {val_accuracy:.2f}%')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('Early stopping triggered!')
            break

writer.close()
print(f"Training completed. Best validation accuracy: {best_val_accuracy:.2f}%")

ModuleNotFoundError: No module named 'tensorboard'

## 4. SRGAN (Generator + Discriminator)
Contains model definitions and training loop skeleton.

In [ ]:
# Cell: SRGAN models skeleton
import torch.nn as nn
# TODO: paste Generator and Discriminator classes (e.g., SRResNet-like) and training loop
# Save checkpoints every n epochs to models/

## 5. Generate synthetic images
Use trained generator to create HR images from LR inputs and save to data/generated_128.

In [ ]:
# Cell: generation script
# TODO: load generator checkpoint and run generation loop saving outputs per class


## 6. Classifier B (train with generated data)
Train classifier on combined real + generated dataset. Save models/classifier_B_best.pth

In [ ]:
# Cell: classifier B skeleton
# TODO: create combined dataset by copying generated images into a copy of processed_128, then train using same recipe as classifier A


## 7. Evaluation
Compute Accuracy, F1, AUC, show confusion matrices, ROC curves, and sample images (LR / Bicubic / GAN / GT).

In [ ]:
# Cell: evaluation
# TODO: implement evaluation code using sklearn.metrics and plotting utilities


## 8. Tips & Next steps
- Set EPOCHS_SR = 150 for final training.
- Use mixed precision (torch.cuda.amp) to speed up.
- Save checkpoints and push to Google Drive if using Colab.

Good luck! If you'd like, I can now fill in the full detailed code for the SRGAN generator/discriminator and the training loops directly in this notebook (long cells). Reply 'yes' to have me expand the SRGAN training code inline in the notebook.